In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
import re
from unidecode import unidecode

In [18]:
# Baixar as stopwords em português
nltk.download('stopwords')
# Obter a lista de stopwords em português
stop_words = stopwords.words('portuguese')

def preprocess_text(text):
    # Remove números e caracteres especiais
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # Converte o texto para minúsculas
    text = text.lower()
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return unidecode(text)


texts = [
    'Investimento de príncipe árabe Neste domingo (18), o Vasco enfrenta o Criciúma pela 23ª rodada do Brasileirão Série A no Heriberto Hülse e uma vitória é importante para o Cruz-Maltino seguir na primeira página da tabela. © Thiago Ribeiro/AGIFPedrinho, presidente do Cruz-Maltino, negocia acordo nos bastidores. Cruzeiro bate Atlético-MG por 2 a 0 e aumenta tabu do clássicoPalmeiras empata com o Cruzeiro em 1 a 1 e é campeão brasileiro O time de Rafael Paiva vem de uma vitória contra o Fluminense e quer seguir neste bom momento. A partida contra o Criciúma vai ter transmissão em tempo real do Bolavip Brasil. Já nos bastidores, o presidente Pedrinho negocia a venda das ações da SAF que pertencem a A-CAP, seguradora que assumiu o controle da 777 Partners. A Crefisa está fora do negócio, mas o Cruz-Maltino tem conversas avançadas com o Fundo de Investimento da Arábia Saudita. Aporte financeiro em R$ 400 milhões O jornalista Fabio Azevedo informou que o novo sócio do Vasco vai precisar assumir o contrato que era com a 777 Partners e compraria a parte que seria aportada pela A-CAP. O comunicador explicou que R$ 310 milhões foram aportados e o novo parceiro do Vascão pagaria esse valor com correção. Dependeria das cifras pedidas pela A-CAP, mas poderia chegar em R$ 500 milhões. Vasco deve vender a SAF aos árabes?Vasco deve vender a SAF aos árabes?0 PESSOAS JÁ VOTARAM No contrato com a 777 Partners, a empresa teria que fazer mais dois aportes financeiros ao Clube Carioca e o Fundo Saudita assumiria esse pagamento. Em setembro de 2024, R$ 280 milhões e no ano que vem em R$ 120 milhões. Venda pode ser confirmada em outubro O líder do fundo de investimento saudita é o príncipe Mohammad bin Salman e os árabes vem fazendo grandes investimentos no futebol. Recentemente, eles compraram o Newcastle, da Inglaterra. Veja tambémAtacante recusa oferta e Vasco de Pedrinho tem negócio com Anderlecht cancelado Além disso, os sauditas estão aportando grandes valores para fazer crescer o futebol local de olho na Copa do Mundo de 2034. A liga adquiriu recentemente contratou nomes como Cristiano Ronaldo e Neymar, além de ter feito oferta ao Vinícius Júnior, do Real Madrid. A ideia dos árabes é tocar a parte financeira do Cruz-Maltino e deixar a gestão do futebol para Pedrinho, segundo o jornalista Fabio Azevedo. Foi especulado que o dono do Chelsea poderia comprar o Vasco, mas o príncipe Mohammad bin Salman é quem tem conversas mais avançadas e acordo pode ser fechado até outubro. O que dizem os torcedores?, ',
    'Criciúma e Vasco se enfrentam hoje (18), às 16h (de Brasília), no Heriberto Hülse, pela 23ª rodada do Campeonato Brasileiro. Onde assistir ao jogo? TV Globo (RJ, SC, ES, PB e as cidades mineiras de Coronel Fabriciano, Juiz de Fora e Montes Claros) e Premiere (pay-per-view) transmitem a partida. O Vasco é o décimo colocado do Brasileirão, com 27 pontos. Já o Criciúma tem 24 pontos, na 15ª posição, e quer se afastar da zona de rebaixamento., ',
    'Convidado do Domingol com Benja deste domingo (18), Euriquinho, filho de Eurico Miranda, ex-presidente do Vasco, compartilhou detalhes inéditos sobre o dia em que o clube entrou em campo com o logo do SBT no uniforme durante uma transmissão exclusiva da Rede Globo. O episódio aconteceu na final da Copa João Havelange, em 2000, no duelo dos vascaínos contra o São Caetano. Leia mais Alexandre Pato, genro de Silvio Santos, presta homenagem ao sogro e faz revelação Fluminense e Corinthians se unem em homenagens a Silvio Santos no Maracanã Neymar presta homenagem a Silvio Santos: &quot;Lenda da TV brasileira&quot; Em entrevista à CNN Brasil, Euriquinho esclareceu que a ideia partiu de seu pai, Eurico, e não do jogador Romário, como muitos especulavam. “Foi ideia do meu pai mesmo. Ele estava numa briga muito grande com a Globo e queria dar uma resposta”, explicou. Operação secreta e reação do SBT O plano foi executado com extremo sigilo, envolvendo apenas quatro ou cinco pessoas. “Foi tudo feito muito rápido. Ele teve a ideia de botar o SBT, botou, e todo mundo ainda questionou se não ia pedir autorização. Ele não pediu, botou na surpresa”, revelou Euriquinho. Contrariando as expectativas de um possível processo judicial, Euriquinho contou que o desfecho foi surpreendentemente positivo. “Dois ou três dias depois, o próprio Silvio [Santos] pegou o telefone, ligou para o meu pai para agradecer muito. Ele falou que tinha sido genial a ideia, que por muitos anos tinha investido em marketing e nunca ninguém tinha promovido tanto a imagem do SBT como o Vasco tinha feito naquele jogo”, relatou. Impacto e legado O episódio, que ocorreu durante a final entre Vasco e São Caetano, não apenas marcou a história do futebol brasileiro, mas também evidenciou as tensões entre clubes e emissoras de televisão na disputa pelos direitos de transmissão. Acompanhe a CNN Esportes em todas as plataformas YouTubeTiktokInstagramTwitter Os textos gerados por inteligência artificial na CNN Brasil são feitos com base nos cortes de vídeos dos jornais de sua programação. Todas as informações são apuradas e checadas por jornalistas. O texto final também passa pela revisão da equipe de jornalismo da CNN. Clique aqui para saber mais. Tópicos CNN Esportes Domingol Futebol brasileiro Silvio Santos Vasco Compartilhe:, ',
    'Criciúma e Vasco se enfrentam neste domingo (18), pela 23ª rodada do Brasileirão, às 16h. O Vasco vai para a partida desfalcado de Pablo Vegetti, principal jogador da equipe, que sentiu uma lesão muscular na coxa contra o Fluminense. Jornalistas da Globo analisaram o confronto e apontaram o favorito a vencer a partida no Estádio Heriberto Hulse. André Loffredo, Alex Escobar, Carlos Mansur, Felipe Diniz, Jéssica Cescon, Rodrigo Coutinho e Tiago Medeiros debateram sobre o jogo e apontaram o Criciúma como o favorito a vencer a partida. No último encontro entre os dois times, o Vasco, ainda sob o comando de Ramón Diaz, levou uma goleada de 4x0 em casa. Uma vitória para cada um dos lados pode mudar o cenário dos times na tabela. Com a soma dos três pontos, o Vasco se firma na primeira parte da tabela e abre larga vantagem da zona de rebaixamento. Uma vitória do Criciúma sobre o Vasco, leva o time a zona de classificação da Sul-Americana, se afastando do fantasma do rebaixamento. Confira os votos:, ',
    'Criciúma e Vasco se enfrentam na tarde deste domingo, dia 18 de agosto, às 16h (horário de Brasília), em jogo válido pela 22ª rodada do Campeonato Brasileiro. A partida ocorre no estádio Heriberto Hülse, na cidade de Criciúma. PUBLICIDADE Após vencer o Fortaleza na última rodada, a equipe comandada por Cláudio Tencati busca se manter fora do Z-4 e ampliar a vantagem para os últimos colocados. O lateral-esquerdo Marcelo Hermes, suspenso após o terceiro cartão amarelo, não joga. Do outro lado do campo, a equipe carioca vem de uma importante vitória contra o Fluminense no clássico dos Gigantes. O time de Rafael Paiva, no entanto, não contará com uma das principais referências do clube. O atacante Pablo Vegetti, com dores na coxa, será poupado. Publicidade, ',
]

vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(texts)

num_clusters = 5  # Escolha o número de clusters desejado
km_model = KMeans(n_clusters=num_clusters)
km_model.fit(tfidf_matrix)

clusters = km_model.labels_.tolist()


# teste2 = preprocess_text(teste1)

# print(teste2)

# print(len(teste1.split(' ')))
# print(len(teste2.split(' ')))



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


InvalidParameterError: The 'stop_words' parameter of TfidfVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'se', 'estas', 'houverei', 'tua', 'essa', 'o', 'serão', 'hajamos', 'tivéramos', 'tínhamos', 'teremos', 'teus', 'houveríamos', 'sejam', 'mais', 'sem', 'estejam', 'aqueles', 'houveremos', 'tenham', 'terei', 'seja', 'isso', 'meus', 'seus', 'tive', 'tivemos', 'hão', 'fui', 'esteja', 'entre', 'meu', 'estivesse', 'na', 'estive', 'nossas', 'estiver', 'às', 'à', 'esses', 'esta', 'dele', 'te', 'de', 'estejamos', 'estivéramos', 'tuas', 'há', 'aquilo', 'for', 'esse', 'estamos', 'era', 'qual', 'quem', 'fora', 'houverem', 'fossem', 'tinham', 'será', 'nosso', 'teria', 'dos', 'tiveram', 'são', 'estivéssemos', 'terão', 'seu', 'as', 'houvera', 'também', 'vos', 'foram', 'nos', 'tenhamos', 'tivéssemos', 'este', 'havemos', 'houvéssemos', 'não', 'você', 'essas', 'eu', 'nossos', 'tiver', 'fosse', 'formos', 'nas', 'nós', 'sejamos', 'eles', 'pelas', 'teu', 'mesmo', 'tivesse

In [24]:
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd

# Passo 1: Baixar as stopwords em português
nltk.download('stopwords')

# Passo 2: Obter a lista de stopwords em português
stop_words = stopwords.words('portuguese')

# Passo 3: Função de pré-processamento dos textos
def preprocess_text(text):
    # Remove números e caracteres especiais
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Converte o texto para minúsculas
    text = text.lower()
    
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

# Suponha que você tenha seus textos em uma lista chamada 'texts_original'
texts_original = [
    "Coração de mãe sempre cabe mais um.",
    "A música acalma a alma.",
    "É importante manter a ação em todas as circunstâncias.",
    "O açúcar está caro.",
    "A saúde é o que mais importa."
]

# Passo 4: Pré-processar os textos
texts = [preprocess_text(text) for text in texts_original]

# Passo 5: Vetorização usando TF-IDF com parâmetros ajustados
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(texts)

# Passo 6: Calcular a densidade de conteúdo
import numpy as np
content_density = np.mean(tfidf_matrix.toarray(), axis=1)

# Passo 7: Criação de um DataFrame para visualização
# Certificando-se de que 'content_density' é 1D
df_density = pd.DataFrame({'Texto': texts_original, 'Densidade de Conteúdo': content_density})

# Passo 8: Ordena o DataFrame pela densidade de conteúdo (do maior para o menor)
df_density_sorted = df_density.sort_values(by='Densidade de Conteúdo', ascending=False)

# Exibir o DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Textos com Maior Densidade de Conteúdo", dataframe=df_density_sorted)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'ace_tools'